This notebook compares the overfitting of Fairlearn Vs OxonFair on a resampled version of the [myocardial infarction dataset](https://archive.ics.uci.edu/dataset/579/myocardial+infarction+complications).

We use sex as the protected attribute.

The initial dataset is balanced, and to induce unfairness in the downstream classifier, we drop half the datapoints that satisfy sex=1  and target_label=0.

Because the dataset is relatively high-dimensional (dims ~= 100) with around 1,000 training points, xgboost overfits perfectly obtaining zero error on the train set.

In [1]:
from oxonfair import FairPredictor, performance, dataset_loader
from oxonfair import group_metrics as gm
import xgboost
import pandas as pd
import numpy as np

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sampler=dataset_loader.resample(1,0,0.5)
train,val,test = dataset_loader.myocardial_infarction(resample=sampler,seed=0)

/Users/chris.russell/code/oxon-fair/src/oxonfair/utils/dataset_loader.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[X.isnull()] = -1
/Users/chris.russell/code/oxon-fair/src/oxonfair/utils/dataset_loader.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[X.isnull()] = -1


We now train XGBoost, and specify a fair predictor over the validation set.

In [3]:
classifier = xgboost.XGBClassifier().fit(X=train['data'], y=train['target'])
fpred=FairPredictor(classifier,val)

We call fit to enforce equal opportunity.

In [4]:
fpred.fit(gm.accuracy,gm.equal_opportunity,0.02)

And evaluate fairness on validation data.

In [5]:
fpred.evaluate_fairness()

,original,updated
Statistical Parity,0.016250,0.014167
Predictive Parity,0.005435,0.084615
Equal Opportunity,0.064935,0.011255
Average Group Difference in False Negative Rate,0.064935,0.011255
Equalized Odds,0.033288,0.016807
Conditional Use Accuracy,0.009843,0.044293
Average Group Difference in Accuracy,0.012500,0.019583
Treatment Equality,0.050000,0.245455


And on the test set.

In [6]:
fpred.evaluate_fairness(test)

,original,updated
Statistical Parity,0.059044,0.064624
Predictive Parity,0.163399,0.108065
Equal Opportunity,0.147059,0.176471
Average Group Difference in False Negative Rate,0.147059,0.176471
Equalized Odds,0.093486,0.107781
Conditional Use Accuracy,0.098748,0.075119
Average Group Difference in Accuracy,0.004747,0.011326
Treatment Equality,0.405983,0.550802


We now check validation performance.

In [7]:
fpred.evaluate()

,original,updated
Accuracy,0.893548,0.903226
Balanced Accuracy,0.789074,0.826993
F1 score,0.713043,0.758065
MCC,0.667074,0.703429
Precision,0.872340,0.839286
Recall,0.602941,0.691176
ROC AUC,0.897302,0.881745


And on the test set.

In [8]:
fpred.evaluate(test)

,original,updated
Accuracy,0.877419,0.874194
Balanced Accuracy,0.757596,0.771390
F1 score,0.660714,0.672269
MCC,0.610892,0.605831
Precision,0.840909,0.784314
Recall,0.544118,0.588235
ROC AUC,0.898396,0.853610


We now run fairlearn on the same data.

In [9]:
from fairlearn.reductions import TruePositiveRateParity, ExponentiatedGradient
mitagator = ExponentiatedGradient(xgboost.XGBClassifier(),TruePositiveRateParity())
mitagator.fit(X=train['data'],y=train['target'],sensitive_features=train['data']['SEX'])

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:214: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.pos_basis[i]["+", e, g] = 1
/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fai

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.TruePositiveRateParity object at 0x3094d83a0>,
                      estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_typ...
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...),
                      nu=0.0)

To evaluate fairlearn, we write a helper function to evaluate performance and fairness on train or test, and concat the outputs together.  

In [10]:
def eval(train, classifier=mitagator):
    return pd.concat((performance.evaluate(train['target'], classifier.predict(train['data'])),
                      performance.evaluate_fairness(train['target'], classifier.predict(train['data']), train['groups'])),axis=0)

out = pd.concat((eval(train), eval(test)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,1.000000,0.877419
Balanced Accuracy,1.000000,0.757596
F1 score,1.000000,0.660714
MCC,1.000000,0.610892
Precision,1.000000,0.840909
Recall,1.000000,0.544118
ROC AUC,1.000000,0.757596
Statistical Parity,0.006611,0.059044
Predictive Parity,0.000000,0.163399
Equal Opportunity,0.000000,0.147059


Evaluating the initially trained baseline classifier we find that, as expected, fairlearn did not alter the performance or unfairness of the classifier.

In [11]:
out = pd.concat((eval(train, classifier), eval(test, classifier)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,1.000000,0.877419
Balanced Accuracy,1.000000,0.757596
F1 score,1.000000,0.660714
MCC,1.000000,0.610892
Precision,1.000000,0.840909
Recall,1.000000,0.544118
ROC AUC,1.000000,0.757596
Statistical Parity,0.006611,0.059044
Predictive Parity,0.000000,0.163399
Equal Opportunity,0.000000,0.147059
